In [1]:
import requests
import json
from fuzzywuzzy import process
import pyaudio
import wave
import pyttsx3
import datetime
import spacy
import re
from pyowm import OWM
import webbrowser
import pyttsx3

def record_audio(RECORD_SECONDS, WAVE_OUTPUT_FILENAME):
    # --------- SETTING PARAMS FOR OUR AUDIO FILE ------------#
    FORMAT = pyaudio.paInt16  # format of wave
    CHANNELS = 2  # no. of audio channels
    RATE = 44100  # frame rate
    CHUNK = 1024  # frames per audio sample
    # --------------------------------------------------------#

    # creating PyAudio object
    audio = pyaudio.PyAudio()

    # open a new stream for microphone
    # It creates a PortAudio Stream Wrapper class object
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    # ----------------- start of recording -------------------#
    print("Listening...")

    # list to save all audio frames
    frames = []

    for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
        # read audio stream from microphone
        data = stream.read(CHUNK)
        # append audio data to frames list
        frames.append(data)

    # ------------------ end of recording --------------------#
    print("Finished recording.")

    stream.stop_stream()  # stop the stream object
    stream.close()  # close the stream object
    audio.terminate()  # terminate PortAudio

    # ------------------ saving audio ------------------------#

    # create wave file object
    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')

    # settings for wave file object
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))

    # closing the wave file object
    waveFile.close()
def read_audio(WAVE_FILENAME):
    # function to read audio(wav) file
    with open(WAVE_FILENAME, 'rb') as f:
        audio = f.read()
    return audio

# Wit speech API endpoint
API_ENDPOINT = 'https://api.wit.ai/speech'

# Wit.ai api access token
wit_access_token = 'SCVCXGT4LDYO2LTM2PWE5F32VBCIPS6E'
def RecognizeSpeech(AUDIO_FILENAME, num_seconds):
    # record audio of specified length in specified audio file
    record_audio(num_seconds, AUDIO_FILENAME)

    # reading audio
    audio = read_audio(AUDIO_FILENAME)

    # defining headers for HTTP request
    headers = {'authorization': 'Bearer ' + wit_access_token,
               'Content-Type': 'audio/wav'}

    # making an HTTP post request
    resp = requests.post(API_ENDPOINT, headers=headers,
                         data=audio)

    # converting response content to JSON format
    data = json.loads(resp.content)

    # get text from data
    text = data['text']

    # return the text
    return text

def fuzzy(sentence, list_of_items):
    str2match=sentence
    strOptions=list_of_items
    Ratios=process.extract(str2match,strOptions)
    highest=process.extractOne(str2match,strOptions)
    return highest
def time_(list_of_text):
    now = datetime.datetime.now()
    return (now.hour,now.minute)

def weather_(text):
    reg_ex = re.search('current weather in (.*)', text)
    if reg_ex:
        city = reg_ex.group(1)
        api_key='ab0d5e80e8dafb2cb81fa9e82431c1fa'
        owm = OWM(api_key)
        obs = owm.weather_at_place(city)
        w = obs.get_weather()
        k = w.get_status()
        x = w.get_temperature(unit='celsius')
    return (city, k, x['temp_max'], x['temp_min'])
def open_web(text):
        reg_ex = re.search('open (.+)',text )
        if reg_ex:
            domain = reg_ex.group(1)
            print(domain)
            url = 'https://www.' + domain+'.com'
            webbrowser.open(url)
        
def intent_finder(list_of_text, list_of_items):
    count=0
    l=list()
    obj=''
    for i in list_of_text:
        if i in list_of_items:
            obj=i
            print("the item you're looking for is:",i)
            test="the item you're looking for is"+""+i
            count=1
            break
    if (count==0):
        for i in list_of_text:
            k=fuzzy(i,list_of_items)
            l.append(k)
        obj = sorted(l, key = lambda i: i[1], reverse = True)[0][0]
        test="Do you mean"+" "+obj
        print("Do you mean : \n",obj)
    return test,obj
        


if __name__ == "__main__":  
    
        text = RecognizeSpeech('myspeech.wav', 5)
    
        print("\nYou said: {}".format(text))
    
        list_of_items=["keys","pen","book","watch","kettle","scissors", "bottle", "wine glass", "cup", "fork", "knife", "bowl","hair dryer","vase","tennis racket","laptop","person","eye glasses","tvmonitor","mouse","chair"]
        list_of_text=text.split()
        sp = spacy.load('en_core_web_sm')
        all_stopwords = sp.Defaults.stop_words
        list_of_text_without_sw= [word for word in list_of_text if not word in all_stopwords]
        print(list_of_text_without_sw)
        test=""
    
        if 'time' in list_of_text_without_sw:
            hour_,minutes_=time_(list_of_text_without_sw)
            print('Current time is %d hours %d minutes' % (hour_,minutes_))
            test='Current time is %d hours %d minutes' % (hour_,minutes_)
        elif 'hello' in list_of_text_without_sw or 'hi' in list_of_text_without_sw :
            print("greetings")
            now = datetime.datetime.now()
            day_time = int(now.strftime('%H'))
            print(day_time)
            if day_time < 12:
                print('Hello Sir.Good morning')
                test= 'Hello Sir. Good morning'
            elif 12 <= day_time < 18:
                print('Hello Sir. Good afternoon')
                test= 'Hello Sir. Good afternoon'
            else:
                print('Hello Sir. Good evening')
                test ='Hello Sir. Good evening'
        elif 'weather' in list_of_text_without_sw:
        
            city, k, temp_max, temp_min = weather_(text)
            test = 'It is '+ str(temp_max)+'celsius and'+' '+ k +' '+ 'in'+' '+city 
        elif 'open' in list_of_text_without_sw:
            open_web(text)
            test='The website you have requested has been opened for you Sir.'
            
    
        else:
            test,obj=intent_finder(list_of_text_without_sw,list_of_items)
            
    
 
   
        engine = pyttsx3.init()
        voices = engine.getProperty('voices')   
        en_voice_id = "com.apple.speech.synthesis.voice.Alex"
        engine.setProperty('voice', en_voice_id)
        print(test)
        engine.say(test)
        engine.runAndWait()

C:\Users\91810\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Listening...
Finished recording.

You said: where is my bottle
['bottle']
the item you're looking for is: bottle
the item you're looking for isbottle


In [ ]:
from ISR.models import RDN, RRDN

#model = RDN(weights='noise-cancel')
model = RRDN(weights='gans')
#model = RDN(weights='psnr-small')
#model = RDN(weights='psnr-large')

In [2]:
import numpy as np
from pil import Image
img = Image.open('C:\\Users\\91810\\Downloads\\YOLO-3-OpenCV\\kb.jpeg')
#img = Image.open('flowers.jpg')

In [ ]:
sr_img = model.predict(np.array(img))
SR= Image.fromarray(sr_img)




In [ ]:
SR.save("SR.jpeg")

In [4]:
import math
foo = Image.open("SR.jpeg")
x, y = foo.size
x2, y2 = math.floor(x*0.25), math.floor(y*0.25)
foo = foo.resize((x2,y2),Image.ANTIALIAS)
foo.save("SR.jpg")




# Detecting Objects on Image with OpenCV deep learning library
#
# Algorithm:
# Reading RGB image --> Getting Blob --> Loading YOLO v3 Network -->
# --> Implementing Forward Pass --> Getting Bounding Boxes -->
# --> Non-maximum Suppression --> Drawing Bounding Boxes with Labels
#
# Result:
# Window with Detected Objects, Bounding Boxes and Labels


# Importing needed libraries
import numpy as np
import cv2
import time
import math

"""
Start of:
Reading input image
"""

# Reading image with OpenCV library
# In this way image is opened already as numpy array
# WARNING! OpenCV by default reads images in BGR format
# Pay attention! If you're using Windows, the path might looks like:
# r'images\woman-working-in-the-office.jpg'
# or:
# 'images\\woman-working-in-the-office.jpg'
#image_BGR = cv2.imread('images/kbottle.jpeg')
image_BGR = cv2.imread('kb.jpeg')
# Showing Original Image
# Giving name to the window with Original Image
# And specifying that window is resizable
cv2.namedWindow('Original Image', cv2.WINDOW_NORMAL)
# Pay attention! 'cv2.imshow' takes images in BGR format
cv2.imshow('Original Image', image_BGR)
# Waiting for any key being pressed
cv2.waitKey(0)
# Destroying opened window with name 'Original Image'
cv2.destroyWindow('Original Image')

# # Check point
# # Showing image shape
#print('Image shape:', image_BGR.shape)  # tuple of (511, 767, 3)

# Getting spatial dimension of input image
h, w = image_BGR.shape[:2]  # Slicing from tuple only first two elements

# # Check point
# # Showing height an width of image
# print('Image height={0} and width={1}'.format(h, w))  # 511 767

"""
End of: 
Reading input image
"""


"""
Start of:
Getting blob from input image
"""

# Getting blob from input image
# The 'cv2.dnn.blobFromImage' function returns 4-dimensional blob
# from input image after mean subtraction, normalizing, and RB channels swapping
# Resulted shape has number of images, number of channels, width and height
# E.G.:
# blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
blob = cv2.dnn.blobFromImage(image_BGR, 1 / 255.0, (416, 416),
                             swapRB=True, crop=True)

# # Check point
# print('Image shape:', image_BGR.shape)  # (511, 767, 3)
# print('Blob shape:', blob.shape)  # (1, 3, 416, 416)

# # Check point
# # Showing blob image in OpenCV window
# # Slicing blob image and transposing to make channels come at the end
#blob_to_show = blob[0, :, :, :].transpose(1, 2, 0)
#print(blob_to_show.shape)  # (416, 416, 3)
#
# # Showing Blob Image
# # Giving name to the window with Blob Image
# # And specifying that window is resizable
#cv2.namedWindow('Blob Image', cv2.WINDOW_NORMAL)
# # Pay attention! 'cv2.imshow' takes images in BGR format
# # Consequently, we DO need to convert image from RGB to BGR firstly
# # Because we have our blob in RGB format
# cv2.imshow('Blob Image', cv2.cvtColor(blob_to_show, cv2.COLOR_RGB2BGR))
# # Waiting for any key being pressed
# cv2.waitKey(0)
# # Destroying opened window with name 'Blob Image'
# cv2.destroyWindow('Blob Image')

"""
End of:
Getting blob from input image
"""


"""
Start of:
Loading YOLO v3 network
"""

# Loading COCO class labels from file
# Opening file
# Pay attention! If you're using Windows, yours path might looks like:
# r'yolo-coco-data\coco.names'
# or:
# 'yolo-coco-data\\coco.names'
with open('yolo-coco-data/coco.names') as f:
#with open('yolo-coco-data/classes.names') as f:
    # Getting labels reading every line
    # and putting them into the list
    labels = [line.strip() for line in f]


# # Check point
# print('List with labels names:')
# print(labels)

# Loading trained YOLO v3 Objects Detector
# with the help of 'dnn' library from OpenCV
# Pay attention! If you're using Windows, yours paths might look like:
# r'yolo-coco-data\yolov3.cfg'
# r'yolo-coco-data\yolov3.weights'
# or:
# 'yolo-coco-data\\yolov3.cfg'
# 'yolo-coco-data\\yolov3.weights'

network = cv2.dnn.readNetFromDarknet('yolo-coco-data/yolov3.cfg',
                                     'yolo-coco-data/yolov3.weights')

##network = cv2.dnn.readNetFromDarknet('yolo-coco-data/yo_test.cfg',
                                     #'yolo-coco-data/win.weights')

# Getting list with names of all layers from YOLO v3 network
layers_names_all = network.getLayerNames()

# # Check point
# print()
# print(layers_names_all)

# Getting only output layers' names that we need from YOLO v3 algorithm
# with function that returns indexes of layers with unconnected outputs
layers_names_output = \
    [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

# # Check point
# print()
# print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']

# Setting minimum probability to eliminate weak predictions
probability_minimum = 0.5

# Setting threshold for filtering weak bounding boxes
# with non-maximum suppression
threshold = 0.3

# Generating colours for representing every detected object
# with function randint(low, high=None, size=None, dtype='l')
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

# # Check point
# print()
# print(type(colours))  # <class 'numpy.ndarray'>
# print(colours.shape)  # (80, 3)
# print(colours[0])  # [172  10 127]

"""
End of:
Loading YOLO v3 network
"""


"""
Start of:
Implementing Forward pass
"""

# Implementing forward pass with our blob and only through output layers
# Calculating at the same time, needed time for forward pass
network.setInput(blob)  # setting blob as input to the network
start = time.time()
output_from_network = network.forward(layers_names_output)
end = time.time()

# Showing spent time for forward pass
#print('Objects Detection took {:.5f} seconds'.format(end - start))

"""
End of:
Implementing Forward pass
"""


"""
Start of:
Getting bounding boxes
"""

# Preparing lists for detected bounding boxes,
# obtained confidences and class's number
bounding_boxes = []
confidences = []
class_numbers = []
inted=str(obj)

Gx = 0
Gy =0
md=0
Ax= []
Ay=[]
Elab=[]
Fstr=[]
j=0

# Going through all output layers after feed forward pass
for result in output_from_network:
    # Going through all detections from current output layer
    for detected_objects in result:
        # Getting 80 classes' probabilities for current detected object
        scores = detected_objects[5:]
        # Getting index of the class with the maximum value of probability
        class_current = np.argmax(scores)
        # Getting value of probability for defined class
        confidence_current = scores[class_current]

        # # Check point
        # # Every 'detected_objects' numpy array has first 4 numbers with
        # # bounding box coordinates and rest 80 with probabilities for every class
        # print(detected_objects.shape)  # (85,)

        # Eliminating weak predictions with minimum probability
        if confidence_current > probability_minimum:
            # Scaling bounding box coordinates to the initial image size
            # YOLO data format keeps coordinates for center of bounding box
            # and its current width and height
            # That is why we can just multiply them elementwise
            # to the width and height
            # of the original image and in this way get coordinates for center
            # of bounding box, its width and height for original image
            box_current = detected_objects[0:4] * np.array([w, h, w, h])

            # Now, from YOLO data format, we can get top left corner coordinates
            # that are x_min and y_min
            x_center, y_center, box_width, box_height = box_current
            
            
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            # Adding results into prepared lists
            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)
            
           

"""
End of:
Getting bounding boxes
"""


"""
Start of:
Non-maximum suppression
"""

# Implementing non-maximum suppression of given bounding boxes
# With this technique we exclude some of bounding boxes if their
# corresponding confidences are low or there is another
# bounding box for this region with higher confidence

# It is needed to make sure that data type of the boxes is 'int'
# and data type of the confidences is 'float'
# https://github.com/opencv/opencv/issues/12789
results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                           probability_minimum, threshold)

"""
End of:
Non-maximum suppression
"""


"""
Start of:
Drawing bounding boxes and labels
"""

# Defining counter for detected objects
counter = 1

# Checking if there is at least one detected object after non-maximum suppression
if len(results) > 0:
    # Going through indexes of results
    for i in results.flatten():
        # Showing labels of the detected objects
        #print('Object {0}: {1}'.format(counter, labels[int(class_numbers[i])]))

        # Incrementing counter
        counter += 1

        # Getting current bounding box coordinates,
        # its width and height
        x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
        box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

        # Preparing colour for current bounding box
        # and converting from numpy array to list
        colour_box_current = colours[class_numbers[i]].tolist()

        # # # Check point
        # print(type(colour_box_current))  # <class 'list'>
        # print(colour_box_current)  # [172 , 10, 127]

        # # Drawing bounding box on the original image
        # cv2.rectangle(image_BGR, (x_min, y_min),
        #               (x_min + box_width, y_min + box_height),
        #               colour_box_current, 2)

        # # Preparing text with label and confidence for current bounding box
        # text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
        #                                        confidences[i])

        # # Putting text with label and confidence on the original image
        # cv2.putText(image_BGR, text_box_current, (x_min, y_min - 5),
        #             cv2.FONT_HERSHEY_COMPLEX, 0.7, colour_box_current, 2)
        box_current = detected_objects[0:4] * np.array([w, h, w, h])
        x_center, y_center, box_width, box_height = box_current
        if (labels[int(class_numbers[i])]==inted):
        
            Gx=x_center
            Gy=y_center
            print(Gx,Gy)
                   
            
        else :
           
            Elab.append(labels[int(class_numbers[i])])
            Ax.append(x_center)
            Ay.append(y_center)
            print(Ax[j],Ay[j])
            j = j+1
            print(j)

    for k in range(j) :

        p= [Gx,Gy]
        q= [Ax[k],Ay[k]]
        d= math.dist(p,q)
                                
        if md==0:
                                    
            md=d
               # print(d)
            Fx=Ax[k]
            Fy=Ay[k]
            obj=Elab[k]
        elif (d<md):
                                    
            md=d
            Fx=Ax[k]
            Fy=Ay[k]
            obj=Elab[k]
                                    
               
               
               
               
               
               
                # print('dist')
                #print(md)
        # if Gx<Fx :
        #         Fstr.append(inted+' is in front of  '+str(obj))

        #         if Gy<Fy:
        #            p=0
        #            q=0
        #            z=p+q
        #            # Fstr.append('  '+str(obj))
        #         elif Gy>Fy:
        #             Fstr.append(' and above of '+str(obj))
        #     elif Gx>Fx:
        #             Fstr.append(inted+' is right of '+str(obj))
        #             if Gy<Fy:
        #                 Fstr.append(' and behind of '+str(obj))
        #             elif Gy>Fy:
        #                 Fstr.append(' and above of '+str(obj))     

# Comparing how many objects where before non-maximum suppression
# and left after

#print(Fstr[1])
#print('Total objects been detected:', len(bounding_boxes))
#print('Number of objects left after non-maximum suppression:', counter - 1)
if len(results) >0:
    # Going through indexes of results
    for i in results.flatten():
        counter += 1

        # Getting current bounding box coordinates,
        # its width and height
        x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
        box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
        if (labels[int(class_numbers[i])]==inted):
       
            cv2.rectangle(image_BGR, (x_min, y_min),
                          (x_min + box_width, y_min + box_height),
                          colour_box_current, 2)

            # Preparing text with label and confidence for current bounding box
            text_box_current = '{}: {:.4f}'.format(inted,
                                                   confidences[i])

            # Putting text with label and confidence on the original image
            cv2.putText(image_BGR, text_box_current, (x_min, y_min - 5),
                        cv2.FONT_HERSHEY_COMPLEX, 0.7, colour_box_current, 2)
        if (labels[int(class_numbers[i])]==obj):
       
            cv2.rectangle(image_BGR, (x_min, y_min),
                          (x_min + box_width, y_min + box_height),
                          colour_box_current, 2)

            # Preparing text with label and confidence for current bounding box
            text_box_current = '{}: {:.4f}'.format(obj,
                                                   confidences[i])

            # Putting text with label and confidence on the original image
            cv2.putText(image_BGR, text_box_current, (x_min, y_min - 5),
                        cv2.FONT_HERSHEY_COMPLEX, 0.7, colour_box_current, 2)
Fstr.append(inted +' is near to '+str(obj))

print(*Fstr,sep='.')
str_mult="the location of the object:"
for i in range(len(Fstr)):
    
    
    str_mult=Fstr[i]+","+str_mult
engine2 = pyttsx3.init()
voices = engine.getProperty('voices')
en_voice_id = "com.apple.speech.synthesis.voice.Alex"
engine2.setProperty('voice', en_voice_id)
print(str_mult)
engine2.say(str_mult)
engine2.runAndWait()

"""
End of:
Drawing bounding boxes and labels
"""


"""
Some comments

With OpenCV function 'cv2.dnn.blobFromImage' we get 4-dimensional
so called 'blob' from input image after mean subtraction,
normalizing, and RB channels swapping. Resulted shape has:
 - number of images
 - number of channels
 - width
 - height
E.G.: blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
"""

# Comparing how many objects where before non-maximum suppression
# and left after
#print()
#print(*Fstr,sep=',')

#print('Total objects been detected:', len(bounding_boxes))
#print('Number of objects left after non-maximum suppression:', counter - 1)

"""
End of:
Drawing bounding boxes and labels
"""


# Showing Original Image with Detected Objects
# Giving name to the window with Original Image
# And specifying that window is resizable
cv2.namedWindow('Detections', cv2.WINDOW_NORMAL)
# Pay attention! 'cv2.imshow' takes images in BGR format
cv2.imshow('Detections', image_BGR)
# Waiting for any key being pressed
cv2.waitKey(0)
# Destroying opened window with name 'Detections'
cv2.destroyWindow('Detections')


"""
Some comments

With OpenCV function 'cv2.dnn.blobFromImage' we get 4-dimensional
so called 'blob' from input image after mean subtraction,
normalizing, and RB channels swapping. Resulted shape has:
 - number of images
 - number of channels
 - width
 - height
E.G.: blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
"""


1264.0694427490234 712.866268157959
1264.0694427490234 712.866268157959
1
1264.0694427490234 712.866268157959
2
1264.0694427490234 712.866268157959
3
laptop is near to book
laptop is near to book,the location of the object:


"\nSome comments\n\nWith OpenCV function 'cv2.dnn.blobFromImage' we get 4-dimensional\nso called 'blob' from input image after mean subtraction,\nnormalizing, and RB channels swapping. Resulted shape has:\n - number of images\n - number of channels\n - width\n - height\nE.G.: blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)\n"

In [5]:
import numpy as np
import cv2
import time
import math

# foo = Image.open("SR.jpeg")
# x, y = foo.size
# x2, y2 = math.floor(x*0.25), math.floor(y*0.3)
# foo = foo.resize((x2,y2),Image.ANTIALIAS)
# foo.save("SR.jpg")



# Detecting Objects on Image with OpenCV deep learning library
#
# Algorithm:
# Reading RGB image --> Getting Blob --> Loading YOLO v3 Network -->
# --> Implementing Forward Pass --> Getting Bounding Boxes -->
# --> Non-maximum Suppression --> Drawing Bounding Boxes with Labels
#
# Result:
# Window with Detected Objects, Bounding Boxes and Labels


# Importing needed libraries
import numpy as np
import cv2
import time
import math

"""
Start of:
Reading input image
"""

# Reading image with OpenCV library
# In this way image is opened already as numpy array
# WARNING! OpenCV by default reads images in BGR format
# Pay attention! If you're using Windows, the path might looks like:
# r'images\woman-working-in-the-office.jpg'
# or:
# 'images\\woman-working-in-the-office.jpg'
#image_BGR = cv2.imread('SR.jpg')
image_BGR = cv2.imread('vv.jpeg')
#image_BGR = cv2.imread('images/woman-working-in-the-office.jpg')
# Showing Original Image
# Giving name to the window with Original Image
# And specifying that window is resizable
cv2.namedWindow('Original Image', cv2.WINDOW_NORMAL)
# Pay attention! 'cv2.imshow' takes images in BGR format
cv2.imshow('Original Image', image_BGR)
# Waiting for any key being pressed
cv2.waitKey(0)
# Destroying opened window with name 'Original Image'
cv2.destroyWindow('Original Image')

# # Check point
# # Showing image shape
# print('Image shape:', image_BGR.shape)  # tuple of (511, 767, 3)

# Getting spatial dimension of input image
h, w = image_BGR.shape[:2]  # Slicing from tuple only first two elements

# # Check point
# # Showing height an width of image
# print('Image height={0} and width={1}'.format(h, w))  # 511 767

"""
End of: 
Reading input image
"""


"""
Start of:
Getting blob from input image
"""

# Getting blob from input image
# The 'cv2.dnn.blobFromImage' function returns 4-dimensional blob
# from input image after mean subtraction, normalizing, and RB channels swapping
# Resulted shape has number of images, number of channels, width and height
# E.G.:
# blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
blob = cv2.dnn.blobFromImage(image_BGR, 1 / 255.0, (416, 416),
                             swapRB=True, crop=False)

# # Check point
# print('Image shape:', image_BGR.shape)  # (511, 767, 3)
# print('Blob shape:', blob.shape)  # (1, 3, 416, 416)

# # Check point
# # Showing blob image in OpenCV window
# # Slicing blob image and transposing to make channels come at the end
# blob_to_show = blob[0, :, :, :].transpose(1, 2, 0)
# print(blob_to_show.shape)  # (416, 416, 3)
#
# # Showing Blob Image
# # Giving name to the window with Blob Image
# # And specifying that window is resizable
# cv2.namedWindow('Blob Image', cv2.WINDOW_NORMAL)
# # Pay attention! 'cv2.imshow' takes images in BGR format
# # Consequently, we DO need to convert image from RGB to BGR firstly
# # Because we have our blob in RGB format
# cv2.imshow('Blob Image', cv2.cvtColor(blob_to_show, cv2.COLOR_RGB2BGR))
# # Waiting for any key being pressed
# cv2.waitKey(0)
# # Destroying opened window with name 'Blob Image'
# cv2.destroyWindow('Blob Image')

"""
End of:
Getting blob from input image
"""


"""
Start of:
Loading YOLO v3 network
"""

# Loading COCO class labels from file
# Opening file
# Pay attention! If you're using Windows, yours path might looks like:
# r'yolo-coco-data\coco.names'
# or:
# 'yolo-coco-data\\coco.names'
with open('yolo-coco-data/coco.names') as f:
#with open('yolo-coco-data/classes.names') as f:
    # Getting labels reading every line
    # and putting them into the list
    labels = [line.strip() for line in f]


# # Check point
# print('List with labels names:')
# print(labels)

# Loading trained YOLO v3 Objects Detector
# with the help of 'dnn' library from OpenCV
# Pay attention! If you're using Windows, yours paths might look like:
# r'yolo-coco-data\yolov3.cfg'
# r'yolo-coco-data\yolov3.weights'
# or:
# 'yolo-coco-data\\yolov3.cfg'
# 'yolo-coco-data\\yolov3.weights'

network = cv2.dnn.readNetFromDarknet('yolo-coco-data/yolov3.cfg',
                                     'yolo-coco-data/yolov3.weights')

##network = cv2.dnn.readNetFromDarknet('yolo-coco-data/yo_test.cfg',
                                     #'yolo-coco-data/win.weights')

# Getting list with names of all layers from YOLO v3 network
layers_names_all = network.getLayerNames()

# # Check point
# print()
# print(layers_names_all)

# Getting only output layers' names that we need from YOLO v3 algorithm
# with function that returns indexes of layers with unconnected outputs
layers_names_output = \
    [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

# # Check point
# print()
# print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']

# Setting minimum probability to eliminate weak predictions
probability_minimum = 0.5

# Setting threshold for filtering weak bounding boxes
# with non-maximum suppression
threshold = 0.3

# Generating colours for representing every detected object
# with function randint(low, high=None, size=None, dtype='l')
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

# # Check point
# print()
# print(type(colours))  # <class 'numpy.ndarray'>
# print(colours.shape)  # (80, 3)
# print(colours[0])  # [172  10 127]

"""
End of:
Loading YOLO v3 network
"""


"""
Start of:
Implementing Forward pass
"""

# Implementing forward pass with our blob and only through output layers
# Calculating at the same time, needed time for forward pass
network.setInput(blob)  # setting blob as input to the network
start = time.time()
output_from_network = network.forward(layers_names_output)
end = time.time()

# Showing spent time for forward pass
print('Objects Detection took {:.5f} seconds'.format(end - start))

"""
End of:
Implementing Forward pass
"""


"""
Start of:
Getting bounding boxes
"""

# Preparing lists for detected bounding boxes,
# obtained confidences and class's number
bounding_boxes = []
confidences = []
class_numbers = []
inted=str(obj1)
Gx = 0
Gy =0
md=0
Ax= []
Ay=[]
Elab=[]
Fstr=[]
j=0
index = 0

# Going through all output layers after feed forward pass
for result in output_from_network:
    # Going through all detections from current output layer
    for detected_objects in result:
        # Getting 80 classes' probabilities for current detected object
        scores = detected_objects[5:]
        # Getting index of the class with the maximum value of probability
        class_current = np.argmax(scores)
        # Getting value of probability for defined class
        confidence_current = scores[class_current]

        # # Check point
        # # Every 'detected_objects' numpy array has first 4 numbers with
        # # bounding box coordinates and rest 80 with probabilities for every class
        # print(detected_objects.shape)  # (85,)

        # Eliminating weak predictions with minimum probability
        if confidence_current > probability_minimum:
            # Scaling bounding box coordinates to the initial image size
            # YOLO data format keeps coordinates for center of bounding box
            # and its current width and height
            # That is why we can just multiply them elementwise
            # to the width and height
            # of the original image and in this way get coordinates for center
            # of bounding box, its width and height for original image
            box_current = detected_objects[0:4] * np.array([w, h, w, h])

            # Now, from YOLO data format, we can get top left corner coordinates
            # that are x_min and y_min
            x_center, y_center, box_width, box_height = box_current
            
            
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            # Adding results into prepared lists
            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)
            
           

"""
End of:
Getting bounding boxes
"""


"""
Start of:
Non-maximum suppression
"""

# Implementing non-maximum suppression of given bounding boxes
# With this technique we exclude some of bounding boxes if their
# corresponding confidences are low or there is another
# bounding box for this region with higher confidence

# It is needed to make sure that data type of the boxes is 'int'
# and data type of the confidences is 'float'
# https://github.com/opencv/opencv/issues/12789
results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                           probability_minimum, threshold)

"""
End of:
Non-maximum suppression
"""


"""
Start of:
Drawing bounding boxes and labels
"""

# Defining counter for detected objects
counter = 1

# # Checking if there is at least one detected object after non-maximum suppression
# if len(results) > 0:
#     # Going through indexes of results
#     for i in results.flatten():
#         # Showing labels of the detected objects
#         #print('Object {0}: {1}'.format(counter, labels[int(class_numbers[i])]))

#         # Incrementing counter
#         counter += 1

#         # Getting current bounding box coordinates,
#         # its width and height
#         x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
#         box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

#         # Preparing colour for current bounding box
#         # and converting from numpy array to list
#         colour_box_current = colours[class_numbers[i]].tolist()

#         # # # Check point
#         # print(type(colour_box_current))  # <class 'list'>
#         # print(colour_box_current)  # [172 , 10, 127]

#         # Drawing bounding box on the original image
#         cv2.rectangle(image_BGR, (x_min, y_min),
#                       (x_min + box_width, y_min + box_height),
#                       colour_box_current, 2)

#         # Preparing text with label and confidence for current bounding box
#         text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
#                                                confidences[i])

#         # Putting text with label and confidence on the original image
#         cv2.putText(image_BGR, text_box_current, (x_min, y_min - 5),
#                     cv2.FONT_HERSHEY_COMPLEX, 0.7, colour_box_current, 2)
#         box_current = detected_objects[0:4] * np.array([w, h, w, h])
#         x_center, y_center, box_width, box_height = box_current
if len(results) >0:
    # Going through indexes of results
    for i in results.flatten():
        counter += 1

        # Getting current bounding box coordinates,
        # its width and height
        x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
        box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
        if (labels[int(class_numbers[i])]==inted):
            index = i
            
       
            cv2.rectangle(image_BGR, (x_min, y_min),
                          (x_min + box_width, y_min + box_height),
                          colour_box_current, 2)

            # Preparing text with label and confidence for current bounding box
            text_box_current = '{}: {:.4f}'.format(inted,
                                                   confidences[i])

            # Putting text with label and confidence on the original image
            cv2.putText(image_BGR, text_box_current, (x_min, y_min - 5),
                        cv2.FONT_HERSHEY_COMPLEX, 0.7, colour_box_current, 2)
        if (labels[int(class_numbers[i])]==obj):
            if (labels[int(class_numbers[index])]==inted):
       
                cv2.rectangle(image_BGR, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              colour_box_current, 2)

                # Preparing text with label and confidence for current bounding box
                text_box_current = '{}: {:.4f}'.format(obj,
                                                       confidences[i])

                # Putting text with label and confidence on the original image
                cv2.putText(image_BGR, text_box_current, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_COMPLEX, 0.7, colour_box_current, 2)
        if (labels[int(class_numbers[i])]==inted):
            
            
        
                    Gx=x_center
                    Gy=y_center
                    #print(Gx,Gy)
            
        else :
           
            Elab.append(labels[int(class_numbers[i])])
            Ax.append(x_min)
            Ay.append(y_min)
            #print(Ax[j],Ay[j])
            j = j+1

        for k in range(j) :

            p= [Gx,Gy]
            q= [Ax[k],Ay[k]]
            d= math.dist(p,q)
                                
            if md==0:
                                    
                md=d
                Fx=Ax[k]
                Fy=Ay[k]
                obj=Elab[k]
            elif (d<md):
                                    
                md=d
                Fx=Ax[k]
                Fy=Ay[k]
                obj=Elab[k]
                                    
                #print('dist')
                #print(md)
#             if Gx<Fx :
#                 Fstr.append(inted+' is left of '+str(obj))

#                 if Gy<Fy:
#                     Fstr.append(' and behind of '+str(obj))
#                 elif Gy>Fy:
#                     Fstr.append(' and above of '+str(obj))
#             elif Gx>Fx:
#                     Fstr.append(inted+' is right of '+str(obj))
#                     if Gy<Fy:
#                         Fstr.append(' and behind of '+str(obj))
#                     elif Gy>Fy:
#                         Fstr.append(' and above of '+str(obj))     

# Comparing how many objects where before non-maximum suppression
# and left after
Fstr.append(inted +' is near to '+str(obj))
if labels[int(class_numbers[index])]==inted:
    print(*Fstr,sep='.')
    
elif labels[int(class_numbers[index])]!=inted:
    print("no object detected or probably the model is  is not trained well ")

#print('Total objects been detected:', len(bounding_boxes))
#print('Number of objects left after non-maximum suppression:', counter - 1)

"""
End of:
Drawing bounding boxes and labels
"""
for i in range(len(Fstr)):

    str_mult=Fstr[i]+","
engine2 = pyttsx3.init()
voices = engine.getProperty('voices')
en_voice_id = "com.apple.speech.synthesis.voice.Alex"
engine2.setProperty('voice', en_voice_id)
engine2.say(str_mult)
engine2.runAndWait()

# Showing Original Image with Detected Objects
# Giving name to the window with Original Image
# And specifying that window is resizable
cv2.namedWindow('Detections', cv2.WINDOW_NORMAL)
# Pay attention! 'cv2.imshow' takes images in BGR format
cv2.imshow('Detections', image_BGR)
# Waiting for any key being pressed
cv2.waitKey(0)
# Destroying opened window with name 'Detections'
cv2.destroyWindow('Detections')


"""
Some comments

With OpenCV function 'cv2.dnn.blobFromImage' we get 4-dimensional
so called 'blob' from input image after mean subtraction,
normalizing, and RB channels swapping. Resulted shape has:
 - number of images
 - number of channels
 - width
 - height
E.G.: blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
"""


Objects Detection took 0.44980 seconds


NameError: name 'obj1' is not defined